In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import (LinearRing, LineString, MultiLineString,
                              MultiPoint, MultiPolygon, Point, Polygon)
import io
# from io import StringIO
# from sqlalchemy import create_engine
import sqlalchemy
import numpy as np
import app.main as m
import app.mydef as md
# imp.reload(md)
mt=md.MyTools()
con = mt.con()
import pdfplumber
import app.netools as ant
nt = ant.NetTools()

In [3]:
liantong = pd.read_excel('g:/1-规划/评估/竞对站点评估/2019年6月/联通工参/输出/带pci汇总/带pci汇总.xlsx')

In [4]:
dianxin = pd.read_excel('g:/1-规划/评估/竞对站点评估/2019年6月/电信工参/输出/带pci汇总/带pci汇总-电信站点.xlsx')

In [11]:
mro = pd.read_sql_table('多天指标汇总-mro-19年8月汇总',con = con )

In [6]:
gongcan = pd.read_csv(open('g:/1-规划/工参/工参数据/ltejizhan20190909.csv'))

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
def juli(df,changjing,pinduan):
    
    F900 = {0:'300',1:'430',2:'430',3:'400',4:'1200',5:'2200'}
    TDD = {'核心城区':'250','一般城区':'400','市郊':'400','县城':'350','乡镇':'800','农村':'1000'}
    
    df['juli']= [F900[cj] if pind == 'FDD900' else TDD[cj] 
                 for cj , pind in zip(df[changjing],df[pinduan])]
    return df

In [8]:
mro.columns

Index(['rq', 'cgi', '小区中文名', 'city', 'vander', 'gsqx', 'atu', 'hd_chun',
       'hd_chte', 'ok_chun', 'ok_chte', 'fgl_cmcc', 'fgl_chun', 'fgl_chte',
       'rsrp_count', 'rsrp_weak', 'un_maxrsrp_count', 'un_maxrsrp_weak113',
       'te_maxrsrp_count', 'te_maxrsrp_weak113', 'un_maxrsrp_weak110',
       'te_maxrsrp_weak110', '工程参数标识', '覆盖类型', '覆盖类别', '覆盖场景', '网络制式(TDD/FDD)',
       '工作频段', '天线频段', 'enbid', 'eci', '覆盖类别2', '通报场景', 'rsrp_avg',
       'rsrp_avg_chun', 'rsrp_avg_chte'],
      dtype='object')

In [9]:
mro["覆盖类别"]=mro["覆盖类别"].fillna('乡镇')#na的用3填充
mro["工作频段"]=mro["工作频段"].fillna('F')#na的用3填充

In [10]:

mro.loc[mro['工作频段'] == '','工作频段']='F'
mro.loc[mro['覆盖类别'] == '','覆盖类别']='乡镇'

In [11]:
mro = juli(mro,'覆盖类别','工作频段')

In [12]:
mro['lon']= 0
mro['lat']= 0

In [13]:
gongcan.columns

Index(['小区CGI(*)', '小区中文名', '网络制式(TDD/FDD)', 'TDL规划站号(新)', '所属地区', '工程参数标识',
       '区县代码', '基站编码', 'VIP基站', '入网工程期', 'ATU网格', 'RRU经度', 'RRU纬度', '天线挂高',
       '地面海拔', '入网时间', '2G共址站名', '与FDD1800共址的FDD900站名', '站址归属单位', '覆盖类型',
       '覆盖类别', '覆盖场景', '覆盖场景细化', '二级覆盖场景', '二级覆盖场景细化', '场景名称', 'VOLTE场景',
       '方位角', '机械下倾角', '电子下倾角', '是否电调天线', '电下倾天线类型', '天线厂家', ' 工作频段', '设备型号',
       '设备类型(大类)', '分布式或一体化', '小区所属空间资源', '基站所属空间资源', 'eNodeB软件版本', '所属OMC',
       '归属区县', '厂家名称', '物理站名', '集中开站所在环节', '集中开站工单号', '姿态仪测量', '姿态仪测量时间',
       '所属eNodeB名称', '采集经度', '采集纬度', 'RRU数量', 'TAC', 'PCI', '中心载频的信道号', '频段指示',
       '空口上下行业务字节数KByte', '统计分类', '集中工参工单号'],
      dtype='object')

In [14]:
gongcan = gongcan.set_index('小区CGI(*)')
mro = mro.set_index('cgi')
mro['lon'].update(gongcan['RRU经度'])
mro['lat'].update(gongcan['RRU纬度'])

In [15]:
gongcan = gongcan.reset_index('小区CGI(*)')
mro = mro.reset_index('cgi')

In [16]:
gongcan.loc[gongcan['小区CGI(*)']=='460-00-184320-65',('RRU经度','RRU纬度')]

,RRU经度,RRU纬度
99069,113.287704,31.865515


In [17]:
mro['juli'] = pd.to_numeric(mro['juli'])

In [18]:
mro_res = mro.loc[(mro['lon']>100) & (mro['lat']>10) & (mro['juli']>10)]

In [19]:
mro_res.columns

Index(['cgi', 'rq', '小区中文名', 'city', 'vander', 'gsqx', 'atu', 'hd_chun',
       'hd_chte', 'ok_chun', 'ok_chte', 'fgl_cmcc', 'fgl_chun', 'fgl_chte',
       'rsrp_count', 'rsrp_weak', 'un_maxrsrp_count', 'un_maxrsrp_weak113',
       'te_maxrsrp_count', 'te_maxrsrp_weak113', 'un_maxrsrp_weak110',
       'te_maxrsrp_weak110', '工程参数标识', '覆盖类型', '覆盖类别', '覆盖场景', '网络制式(TDD/FDD)',
       '工作频段', '天线频段', 'enbid', 'eci', '覆盖类别2', '通报场景', 'rsrp_avg',
       'rsrp_avg_chun', 'rsrp_avg_chte', 'juli', 'lon', 'lat'],
      dtype='object')

In [20]:
mro_res = mro_res[['cgi','juli', 'lon', 'lat']]

In [21]:
mro_tu = nt.add_buffer_df(mro_res,lon='lon', lat='lat', buff_col='juli')

In [22]:
# dianxin.info()

In [23]:
# liantong['经度'] = liantong['经度'].str.replace(' ','')#替换表中的换行 \r符号
# liantong['经度'] = pd.to_numeric(liantong['经度'])
# liantong['纬度'] = pd.to_numeric(liantong['纬度'])

In [24]:
# dianxin['经度'] = dianxin['经度'].str.replace(' ','').str.replace(' ','')#替换表中的换行 \r符号
# dianxin['经度'] = pd.to_numeric(dianxin['经度'])

In [25]:
# dianxin['纬度'] = dianxin['纬度'].str.replace(' ','').str.replace(',','')#替换表中的换行 \r符号
# dianxin['纬度'] = pd.to_numeric(dianxin['纬度'])

In [26]:
liantong_tu = nt.add_points(liantong, coords=['经度', '纬度', '', ''])
dianxin_tu = nt.add_points(dianxin, coords=['经度', '纬度', '', ''])

In [27]:
liantong_tu

,地市,经度,纬度,pci,EARFCN,站名,LTE站号,ChBw,Cell,区县,站型,geometry
0,十堰市,109.647541,33.240996,NaN,1850.0,LF-郧西新川,NaN,NaN,NaN,NaN,宏站,POINT (109.647541 33.240996)
1,十堰市,110.993611,33.236389,NaN,1850.0,LF-郧县白浪(拉远小白浪移动),NaN,NaN,NaN,NaN,宏站,POINT (110.993611 33.236389)
2,十堰市,110.994604,33.234430,NaN,2145.0,LF-郧县白浪,NaN,NaN,NaN,NaN,宏站,POINT (110.994604 33.23443)
3,十堰市,110.992692,33.231142,NaN,2145.0,LF-郧县白浪(拉远程家台),NaN,NaN,NaN,NaN,宏站,POINT (110.992692 33.231142)
4,十堰市,110.955610,33.228011,NaN,1850.0,LF-郧县鲍鱼岭(拉远高化坪),NaN,NaN,NaN,NaN,宏站,POINT (110.95561 33.228011)
5,十堰市,110.999948,33.227141,NaN,951.0,LF-郧县白浪GL,NaN,NaN,NaN,NaN,宏站,POINT (110.999948 33.227141)
6,十堰市,109.542180,33.222494,NaN,951.0,LF-郧西湖北口GL,NaN,NaN,NaN,NaN,宏站,POINT (109.54218 33.222494)
7,十堰市,109.621928,33.216694,NaN,951.0,LF-郧西虎坪GL,NaN,NaN,NaN,NaN,宏站,POINT (109.621928 33.216694)
8,十堰市,109.583917,33.214991,NaN,2145.0,LF-郧西虎新路(广电),NaN,NaN,NaN,NaN,宏站,POINT (109.583917 33.214991)
9,十堰市,111.008690,33.208880,NaN,2145.0,LF-郧县白浪郭村,NaN,NaN,NaN,NaN,室分,POINT (111.00869 33.20888)


In [28]:
liantong_res = gpd.sjoin(mro_tu,liantong_tu)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2287: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [29]:
dianxin_res = gpd.sjoin(mro_tu,dianxin_tu)

In [32]:
liantong_res = liantong_res[['cgi', 'juli', 'lon', 'lat', 'index_right', '地市', '经度',
       '纬度', 'pci', 'EARFCN', '站名', 'LTE站号', 'ChBw', 'Cell', '区县', '站型']]

In [34]:
dianxin_res = dianxin_res[['cgi', 'juli', 'lon', 'lat', 'index_right', '地市', '经度',
       '纬度', 'pci', 'EARFCN', '站名', 'LTE站号', 'ChBw', 'Cell', '区县', '站型']]

In [35]:
liantong_res.to_csv('liantong_res.csv')

In [36]:
dianxin_res.to_csv('dianxin_res.csv')

In [37]:
display(liantong_res.shape ,dianxin_res.shape)

(1825536, 16)

(1593704, 16)

## 读取结果 

In [19]:
liantong_res = pd.read_csv('liantong_res.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (11,12,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
dianxin_res = pd.read_csv('dianxin_res.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
mro['liantong']=0
mro['dianxin']=0

In [23]:
liantong_res_g = liantong_res.groupby(by='cgi').agg({'地市':'count'})
dianxin_res_g = dianxin_res.groupby(by='cgi').agg({'地市':'count'})

In [28]:
mro = mro.set_index('cgi')

In [30]:
mro['liantong'].update(liantong_res_g['地市'])

In [31]:
mro['dianxin'].update(dianxin_res_g['地市'])

In [33]:
mro.to_csv('mro.csv')

In [3]:
mro_daoru = pd.read_csv('mro.csv')

In [4]:
mro_daoru

,cgi,rq,小区中文名,city,vander,gsqx,atu,hd_chun,hd_chte,ok_chun,...,天线频段,enbid,eci,覆盖类别2,通报场景,rsrp_avg,rsrp_avg_chun,rsrp_avg_chte,liantong,dianxin
0,460-00-1387-1,2019年08,NaN,NaN,NaN,NaN,NaN,f,f,t,...,NaN,1387,355073,全网,NaN,-90.586700,-109.271982,-105.683887,0,0
1,460-00-184320-65,2019年08,D_随县预制板厂-HLH-1,随州,华为,随州市,NaN,f,f,t,...,NaN,184320,47185985,全网,NaN,-94.870797,-98.170362,-96.635937,0,0
2,460-00-184320-66,2019年08,D_随县预制板厂-HLH-2,随州,华为,随州市,NaN,f,f,t,...,NaN,184320,47185986,全网,NaN,-92.046270,-99.810412,-102.303077,0,0
3,460-00-184320-67,2019年08,D_随县预制板厂-HLH-3,随州,华为,随州市,NaN,f,f,t,...,NaN,184320,47185987,全网,NaN,-89.843917,-95.312825,-93.555979,0,0
4,460-00-184321-65,2019年08,曾都何店湖北金龙集团-HLH-1,随州,华为,随州市,NaN,f,f,t,...,NaN,184321,47186241,全网,NaN,-82.898706,-103.507273,-100.888354,0,0
5,460-00-184321-66,2019年08,曾都何店湖北金龙集团-HLH-2,随州,华为,随州市,NaN,f,f,t,...,NaN,184321,47186242,全网,NaN,-90.604059,-101.830120,-105.698342,0,0
6,460-00-184321-67,2019年08,曾都何店湖北金龙集团-HLH-3,随州,华为,随州市,NaN,f,f,t,...,NaN,184321,47186243,全网,NaN,-89.126543,-104.636916,-100.602245,0,0
7,460-00-184323-65,2019年08,曾都蒋家岗2-HLH-1,随州,华为,随州市,NaN,f,f,t,...,NaN,184323,47186753,全网,NaN,-98.889529,-102.236884,-127.087820,2,1
8,460-00-184323-66,2019年08,曾都蒋家岗2-HLH-2,随州,华为,随州市,NaN,f,f,t,...,NaN,184323,47186754,全网,NaN,-93.162666,-105.575364,-93.201933,2,1
9,460-00-184323-67,2019年08,曾都蒋家岗2-HLH-3,随州,华为,随州市,NaN,f,f,t,...,NaN,184323,47186755,全网,NaN,-87.321491,-114.255931,-97.691271,2,1


In [6]:
mro_daoru.to_csv('mro.csv',encoding = 'gbk')